### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [2]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [3]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [4]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [5]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42617361,2927,79085,51530,0,0,1133,5,2025-03-27 22:00,2025-03-27 22:00,0,11,1113,2,1
34,42613989,79085,89509,1001,3,77,1133,6,2025-03-28 22:00,2025-03-28 22:00,0,1,15303,4,1
35,42617375,142694,79085,51530,0,0,1134,0,2025-03-29 22:00,2025-03-29 22:00,3,0,1756,2,1
36,42410265,79085,23775,1349,3,70,1134,1,2025-03-30 21:59,2025-03-30 21:59,2,4,9319,2,1
37,42617787,79085,60678,51530,0,0,1134,2,2025-03-31 22:00,2025-03-31 22:00,1,0,2098,3,1
38,42624584,86975,79085,51530,0,0,1134,3,2025-04-01 22:00,2025-04-01 22:00,2,1,2858,3,1
39,42410271,23755,79085,1349,4,70,1134,4,2025-04-02 21:59,2025-04-02 21:59,0,1,21579,6,1


In [6]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [7]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [8]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [9]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,2927,45927,32569,0,1,0,0,0,4-3-3 - wlasne,4,5,5,42617361
1,79085,24102,37460,0,31,1,1,0,4-2-4a,11,9,8,42617361
0,79085,38027,34946,0,3,2,1,0,5-4-1aSR,5,7,7,42613989
1,89509,32323,35404,0,9,2,0,0,CAP2ATT,8,9,6,42613989
0,142694,30117,34100,0,9,5,0,0,22V LAT EDI,9,9,9,42617375
1,79085,39831,35848,2,4,3,0,0,4-3-3a,9,9,10,42617375
0,79085,48679,34028,3,5,0,0,0,3-4-3SP,9,9,11,42410265
1,23775,21276,35927,0,13,3,0,0,serie D,9,10,8,42410265
0,79085,26248,34618,0,17,3,0,0,4-4-2c,10,10,9,42617787
1,60678,43714,35344,0,5,2,0,0,Sweep,8,7,9,42617787


In [10]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [11]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1658

### Estatísticas da Liga

In [12]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [13]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [14]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,4,12,4,0,0,17,3,12514017004001,1349
1,10520,4,12,4,0,0,16,3,12513016004011,1349
2,124139,4,10,3,1,0,12,2,10510012003006,1349
3,78671,4,9,3,0,1,14,2,9512014003008,1349
4,79085,4,9,3,0,1,12,4,9508012003004,1349
5,23755,4,6,2,0,2,15,6,6509015002010,1349
6,79866,4,6,2,0,2,14,5,6509014002009,1349
7,122169,4,4,1,1,2,3,6,4497003001005,1349
8,79665,4,3,1,0,3,1,21,3480001001002,1349
9,89569,4,0,0,0,4,2,9,493002000007,1349


In [15]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [16]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [17]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [18]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [19]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [20]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()